In [1]:
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
#! wget -o  ../../datasets/ https://archive.ics.uci.edu/ml/machine-learning-databases/00280/HIGGS.csv.gz
column_names = 'label, lepton pT, lepton eta, lepton phi, missing energy magnitude, missing energy phi, jet 1 pt, jet 1 eta, jet 1 phi, jet 1 b-tag, jet 2 pt, jet 2 eta, jet 2 phi, jet 2 b-tag, jet 3 pt, jet 3 eta, jet 3 phi, jet 3 b-tag, jet 4 pt, jet 4 eta, jet 4 phi, jet 4 b-tag, m_jj, m_jjj, m_lv, m_jlv, m_bb, m_wbb, m_wwbb'.split(', ')
data = pd.read_csv('../../datasets/HIGGS.csv.gz', header=None, names=column_names)

In [3]:
X_data = data.drop('label', 1)
X_data = StandardScaler().fit_transform(X_data)

labels = data["label"]

In [111]:
from sklearn.model_selection import train_test_split
import numpy as np
indx_train, indx_test = train_test_split(np.arange(len(labels), dtype='int32'), stratify=labels, train_size=4e-4, test_size=0.1)

In [112]:
y_train = labels[indx_train]
y_test = labels[indx_test]

X_train = X_data[indx_train]
X_test = X_data[indx_test]

In [113]:
X_good = X_train[np.where(y_train==0)[0]]
X_bad = X_train[np.where(y_train==1)[0]]

In [114]:
X_good.shape, X_bad.shape, X_train.shape, X_data.shape

((2068, 28), (2332, 28), (4400, 28), (11000000, 28))

In [115]:
import theano
import theano.tensor as T
import lasagne
from lasagne.layers import InputLayer, DenseLayer
from lasagne.nonlinearities import softmax
from lasagne.objectives import categorical_crossentropy

input_size = X_good.shape[1] # features cnt

input_var = T.matrix('input', dtype='float32')
weights_var = T.vector('weights', dtype='float32')
target_var = T.matrix('target', dtype='float32')
lr_var = T.scalar('learning rate')

network = InputLayer(shape=(None, input_size), input_var=input_var)
network = DenseLayer(network, 400)
network = DenseLayer(network, 2, nonlinearity=softmax)
output = lasagne.layers.get_output(network)

loss = (weights_var * categorical_crossentropy(output, target_var)).mean()
params = lasagne.layers.get_all_params(network, trainable=True)
updates = lasagne.updates.rmsprop(loss, params, learning_rate=lr_var)

train_fn = theano.function([input_var, weights_var, target_var, lr_var], [output, loss], updates=updates, allow_input_downcast=True)
predict_fn = theano.function([input_var], [output], allow_input_downcast=True)

In [116]:
X_min, X_max = np.min(X_data, axis=0), np.max(X_data, axis=0)

def random_sampling(size, features_range=[X_min, X_max]):
    return np.array([features_range[0][i] * np.ones(size) + np.random.rand(size) * (features_range[1][i] - features_range[0][i]) for i in range(len(features_range[0]))]).transpose()

In [117]:
import sys
sys.path.append('../../')
from hmc import *

In [ ]:
# from sklearn.metrics import *
from evaluation import get_anomaly_metrics
from keras.utils import to_categorical

lr = 1e-3
epoches = 5000
batch_size = 10000
sampler_batch_size = 10000
sampler_epoch_rate = 1
fraction = 10
alpha = 1e-3

true_bad_size = int(alpha * fraction * y_train.sum())
fake_bad_size = int((1.0 - alpha) * fraction * y_train.sum())
good_size = X_good.shape[0]

X_bad_true = X_train[y_train == 1][np.random.choice(np.arange(int(y_train.sum())), size=true_bad_size), :]
X_bad_fake = random_sampling(sampler_batch_size)


target = np.concatenate([np.zeros(good_size), np.ones(true_bad_size + fake_bad_size)])
good_weights = np.ones(X_good.shape[0])
true_bad_weights = alpha * np.ones(true_bad_size)
fake_bad_weights = (1 - alpha) * np.ones(fake_bad_size)

position = theano.shared(X_bad_fake)

def P(x):
    preds = x
    for layer in lasagne.layers.get_all_layers(network)[1:]: # pass x variable through all network layers except input one
        preds = layer.get_output_for(preds)
    preds = preds[:, 1]
    return 1 - T.exp(-preds / (1 - preds))

def get_P(x):
    preds = predict_fn(x)[0].squeeze()[:, 1]
    return 1 - np.exp(-preds / (1 - preds))

sampler = HMC_sampler.new_from_shared_positions(position, P,
                      initial_stepsize=1e-3, stepsize_max=0.5)

In [ ]:
aucs = [0.4, 0.41]
for epoch in range(epoches):
    if epoch % 150 == 0 and epoch > 0: # and aucs[-1] - np.mean(aucs[-6:-1]) < 1e-4:
        lr /= 3
    print('Epoch: %d' % epoch)
    
    [sampler.draw() for _ in range(sampler_epoch_rate)]
    
    X_bad_fake = np.concatenate([sampler.draw() for _ in range(int(round(fake_bad_size / sampler_batch_size)))][:fake_bad_size])
    X_mix = np.concatenate([X_good, X_bad_fake, X_bad_true])
    
    eps = 1e-4
    fake_bad_weights = 1./(eps + get_P(X_bad_fake))
#     fake_bad_weights /= max(fake_bad_weights)
    fake_bad_weights *= (1. - alpha)
    
    
    weights = np.concatenate([good_weights, fake_bad_weights, true_bad_weights])
    indices = np.arange(len(weights))
    np.random.shuffle(indices)
    for i in range(int(len(indices)/batch_size)):
        batch_idx = indices[range(batch_size*i,min(batch_size*(i+1), len(indices)))]
        _, loss_value = train_fn(X_mix[batch_idx], weights[batch_idx], to_categorical(list(target[batch_idx]) + [0,1])[:-2], lr)
    y_pred = predict_fn(X_test)[0].squeeze()[:, 1]
    metrics = get_anomaly_metrics(y_test, y_pred)
    aucs.append(metrics['PR_AUC'])
    print(metrics)
    
    print('Learning rate: ', lr)

Epoch: 0
{'Precision@0.9': 0.5290756797124834, 'Precision@0.8': 0.5261848593873221, 'PR_AUC': 0.4923839694931823, 'Precision@0.95': 0.5298331981264924, 'P@10': 1.0, 'Precision@0.99': 0.5298942778319997, 'ROC_AUC': 0.4640716268741413}
('Learning rate: ', 0.001)
Epoch: 1
{'Precision@0.9': 0.5297519561452497, 'Precision@0.8': 0.527163064291156, 'PR_AUC': 0.49277273458499044, 'Precision@0.95': 0.5299023781455122, 'P@10': 1.0, 'Precision@0.99': 0.5299023781455122, 'ROC_AUC': 0.4651864996971379}
('Learning rate: ', 0.001)
Epoch: 2
{'Precision@0.9': 0.5299105031469422, 'Precision@0.8': 0.5279996288442346, 'PR_AUC': 0.4931599920506903, 'Precision@0.95': 0.5299105031469422, 'P@10': 1.0, 'Precision@0.99': 0.5299105031469422, 'ROC_AUC': 0.4661721209224568}
('Learning rate: ', 0.001)
Epoch: 3
{'Precision@0.9': 0.5299155482993693, 'Precision@0.8': 0.5287033793290319, 'PR_AUC': 0.493535044072326, 'Precision@0.95': 0.5299155482993693, 'P@10': 1.0, 'Precision@0.99': 0.5299155482993693, 'ROC_AUC': 0.46

{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5114781916700494, 'Precision@0.95': 0.529919572654157, 'P@10': 0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.49079829221979543}
('Learning rate: ', 0.001)
Epoch: 33
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5125595757743455, 'Precision@0.95': 0.529919572654157, 'P@10': 0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.49188465752432353}
('Learning rate: ', 0.001)
Epoch: 34
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5136219890330078, 'Precision@0.95': 0.529919572654157, 'P@10': 0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.49292787084631806}
('Learning rate: ', 0.001)
Epoch: 35
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5147238786259812, 'Precision@0.95': 0.529919572654157, 'P@10': 0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.49403315935451353}
('Learnin

{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5130830038113461, 'Precision@0.95': 0.529919572654157, 'P@10': 0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.4933806684570819}
('Learning rate: ', 0.001)
Epoch: 65
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5115275772841987, 'Precision@0.95': 0.529919572654157, 'P@10': 0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.49220152129044037}
('Learning rate: ', 0.001)
Epoch: 66
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5104918883601742, 'Precision@0.95': 0.529919572654157, 'P@10': 0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.49118800780266997}
('Learning rate: ', 0.001)
Epoch: 67
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5090584382722446, 'Precision@0.95': 0.529919572654157, 'P@10': 0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.4889684510495901}
('Learning 

{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5050186144828825, 'Precision@0.95': 0.529919572654157, 'P@10': 1.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.48404393756281927}
('Learning rate: ', 0.001)
Epoch: 97
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5052146821292052, 'Precision@0.95': 0.529919572654157, 'P@10': 0.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.483970966097787}
('Learning rate: ', 0.001)
Epoch: 98
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5057389405950496, 'Precision@0.95': 0.529919572654157, 'P@10': 1.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.484529830918384}
('Learning rate: ', 0.001)
Epoch: 99
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5062407127135401, 'Precision@0.95': 0.529919572654157, 'P@10': 1.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.4850653612931438}
('Lear

{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5183570187291455, 'Precision@0.95': 0.529919572654157, 'P@10': 1.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.49387187736074695}
('Learning rate: ', 0.001)
Epoch: 129
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5188869794646543, 'Precision@0.95': 0.529919572654157, 'P@10': 1.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.4947386288201153}
('Learning rate: ', 0.001)
Epoch: 130
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.5200731393923415, 'Precision@0.95': 0.529919572654157, 'P@10': 1.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.496543959433863}
('Learning rate: ', 0.001)
Epoch: 131
{'Precision@0.9': 0.529919572654157, 'Precision@0.8': 0.529919572654157, 'PR_AUC': 0.522128114016117, 'Precision@0.95': 0.529919572654157, 'P@10': 1.0, 'Precision@0.99': 0.529919572654157, 'ROC_AUC': 0.4975950987374149}
('L

...

{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459785759429708, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194857846178835}
('Learning rate: ', 1.5241579027587256e-07)
Epoch: 1285
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459786164895883, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194858496291899}
('Learning rate: ', 1.5241579027587256e-07)
Epoch: 1286
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459786921155312, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.519485940519943}
('Learning rate: ', 1.5241579027587256e-07)
Epoch: 1287
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459787556499146, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.

{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459802991145378, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194882661252159}
('Learning rate: ', 1.5241579027587256e-07)
Epoch: 1315
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459803538864657, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194883312476642}
('Learning rate: ', 1.5241579027587256e-07)
Epoch: 1316
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459804057870049, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194884149473784}
('Learning rate: ', 1.5241579027587256e-07)
Epoch: 1317
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.545980461831429, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.

{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459817029077092, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194905258084831}
('Learning rate: ', 1.5241579027587256e-07)
Epoch: 1345
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459817620717367, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194906075192571}
('Learning rate: ', 1.5241579027587256e-07)
Epoch: 1346
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459817989570634, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194906858659185}
('Learning rate: ', 1.5241579027587256e-07)
Epoch: 1347
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459818432532584, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0

{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459822079730503, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.519491509403292}
('Learning rate: ', 5.0805263425290854e-08)
Epoch: 1375
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459822081308007, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194915220933609}
('Learning rate: ', 5.0805263425290854e-08)
Epoch: 1376
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459822117119824, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194915411832058}
('Learning rate: ', 5.0805263425290854e-08)
Epoch: 1377
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459822149726742, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.

{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459824088013885, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194920489352618}
('Learning rate: ', 5.0805263425290854e-08)
Epoch: 1405
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459824103030865, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194920665935344}
('Learning rate: ', 5.0805263425290854e-08)
Epoch: 1406
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459824109863013, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0.99': 0.529921072296758, 'ROC_AUC': 0.5194920804381509}
('Learning rate: ', 5.0805263425290854e-08)
Epoch: 1407
{'Precision@0.9': 0.529921072296758, 'Precision@0.8': 0.529921072296758, 'PR_AUC': 0.5459824198728435, 'Precision@0.95': 0.529921072296758, 'P@10': 0.75, 'Precision@0

In [ ]:
import evaluation
reload(evaluation)
from evaluation import get_anomaly_metrics

In [ ]:
Score = y_pred

precision, recall, _ = precision_recall_curve(y_test, Score)
pr_auc = auc(recall, precision)

plt.figure()
plt.title('(1 + e) with Hamilton sampling')
plt.plot(recall, precision, label='PR curve (area = %0.3f)' % pr_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()

fpr, tpr, _ = roc_curve(y_test, Score)
roc_auc = auc(fpr, tpr)

plt.figure()
plt.title('(1 + e) with Hamilton sampling')
plt.plot(fpr, tpr, label='ROC curve (area = %0.3f)' % roc_auc)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.legend(loc="lower right")
plt.grid(True)
plt.show()